## Installing all dependencies

In [7]:
##
%pip install opencv-python==4.1.1.26

# install lxml(helps to parse xml annotations)
%pip install lxml

# install tqdm(it helps to visualize the progress bar while iterating a loop)
# it adds a smart progress meter in loop execution
%pip install tqdm
%pip install numpypip
%pip install pandas 

ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.1.26 (from versions: 3.4.0.14, 3.4.8.29, 3.4.9.31, 3.4.9.33, 3.4.10.35, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.14.51, 3.4.14.53, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.1.2.30, 4.2.0.32, 4.2.0.34, 4.3.0.36, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.2.52, 4.5.2.54, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68)
ERROR: No matching distribution found for opencv-python==4.1.1.26
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install opencv-python==4.1.1.26

In [2]:
!pip install sklearn

  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=c346ef99c321b0278fe33e484e56d4f70a0ddaf81fad3277ce1ae538849f4917
  Stored in directory: /home/jupyter/.cache/pip/wheels/15/9a/b1/2478e73a520d596fab614693f5cd1beef4ba3db737bed1ac7d
Successfully built sklearn


In [12]:
%pip install absl-py

Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -V

Python 3.7.12


In [3]:
import zipfile
import os
import random
import numpy as np
import pandas as pd
import shutil

from absl import logging
from absl.flags import FLAGS

import time
#hashlib helps to encypt the data so that data does not get changes while parsing
import hashlib

import lxml.etree # to process xml tree structure in Pascal Voc annot files
import tqdm

from IPython.display import Image
import cv2

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Add,
    Concatenate,
    Conv2D,
    Input,
    Lambda,
    LeakyReLU,
    MaxPool2D,
    UpSampling2D,
    ZeroPadding2D,
    BatchNormalization,
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import (
    binary_crossentropy,
    sparse_categorical_crossentropy
)

from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)

#flags can be used to define or initialize the configuration variable throughout the pipeline
from absl import app, flags, logging

#from .utils import broadcast_iou

from pathlib import Path
import xml.etree.cElementTree as ET
from PIL import Image
     

2023-01-13 08:18:46.446733: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-13 08:18:47.372463: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-01-13 08:18:47.372566: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [4]:
# system setup 
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

2023-01-13 08:19:13.667109: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-13 08:19:15.660332: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 08:19:15.776352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 08:19:15.778172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12727692605978264480
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14242217984
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12971501120987339820
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [5]:
# checking GPU utilization
device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
  raise SystemError('GPU device not found!')

print(f'Found GPU at {device}')

Found GPU at /device:GPU:0


2023-01-13 08:19:39.361730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 08:19:39.363799: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 08:19:39.365567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 08:19:39.367434: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 08:19:39.369154: I tensorflow/compiler/xla/stream_executo

In [7]:
!wget https://pjreddie.com/media/files/yolov3.weights ./


--2023-01-13 08:20:31--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  46.0MB/s    in 5.5s    

2023-01-13 08:20:37 (43.0 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2023-01-13 08:20:37--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2023-01-13 08:20:37--
Total wall clock time: 5.9s
Downloaded: 1 files, 237M in 5.5s (43.0 MB/s)


In [8]:
! git clone https://github.com/WongKinYiu/yolov7.git


Cloning into 'yolov7'...
remote: Enumerating objects: 1127, done.
remote: Total 1127 (delta 0), reused 0 (delta 0), pack-reused 1127
Receiving objects: 100% (1127/1127), 69.92 MiB | 39.38 MiB/s, done.
Resolving deltas: 100% (528/528), done.


In [10]:
import os

os.chdir("./yolov7")

In [13]:
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 54.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 704.2 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.7.0a0+a6073f0
    Uninstalling torchvision-0.7.0a0+a6073f0:
      Successfully uninstalled torchvision-0.7.0a0+a6073f0


In [15]:
! python3 detect.py --source ../lyw_cam1_with_pii_H265.mp4 --weights yolov7.pt --name video_1 --view-img


Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='video_1', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='../lyw_cam1_with_pii_H265.mp4', update=False, view_img=True, weights=['yolov7.pt'])
YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.9.1+cu111 CUDA:0 (Tesla T4, 14910.375MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36905341 parameters, 6652669 gradients, 104.5 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

video 1/1 (1/2512) /home/jupyter/video_analytics/yolov7/../lyw_cam1_with_pii_H265.mp4: 3 persons, 2 benchs, Done. (19.6ms) Inference, (1.4ms) NMS
: cannot connect to X server 


In [16]:
!python3 detect.py --weights yolov7.pt --conf 0.60 --img-size 640 --source ../lyw_cam1_with_pii_H265.mp4

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.6, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='../lyw_cam1_with_pii_H265.mp4', update=False, view_img=False, weights=['yolov7.pt'])
YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.9.1+cu111 CUDA:0 (Tesla T4, 14910.375MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36905341 parameters, 6652669 gradients, 104.5 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

video 1/1 (1/2512) /home/jupyter/video_analytics/yolov7/../lyw_cam1_with_pii_H265.mp4: 2 persons, 1 bench, Done. (19.6ms) Inference, (1.3ms) NMS
video 1/1 (2/2512) /home/jupyter/video_analytics/yolov7/../lyw_cam1_with_pii_H265.mp4: 1 person, 1 bench, Done. (15.9ms) Inference, (1.0ms) NMS
video 1/1 (3/2512) /home/jupyter/video_analytics/yo

## Main code

In [2]:
import zipfile
import os
import random
import numpy as np
import pandas as pd
import shutil

from absl import logging
from absl.flags import FLAGS

import time
#hashlib helps to encypt the data so that data does not get changes while parsing
import hashlib

import lxml.etree # to process xml tree structure in Pascal Voc annot files
import tqdm

from IPython.display import Image
import cv2

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Add,
    Concatenate,
    Conv2D,
    Input,
    Lambda,
    LeakyReLU,
    MaxPool2D,
    UpSampling2D,
    ZeroPadding2D,
    BatchNormalization,
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import (
    binary_crossentropy,
    sparse_categorical_crossentropy
)

from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)

#flags can be used to define or initialize the configuration variable throughout the pipeline
from absl import app, flags, logging

#from .utils import broadcast_iou

from pathlib import Path
import xml.etree.cElementTree as ET
from PIL import Image

2023-01-14 01:13:50.571496: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-14 01:13:52.543423: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-01-14 01:13:52.543581: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

### Environment Setup
Improve Reproducibility



In [3]:
# improve reproducibility and make it more deterministic
os.environ['TF_CUDNN_DETEREMINISTIC'] = '1'
random.seed(hash('setting random seeds') % 2**32 -1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash('by removing stochasticity')% 2**32 -1)


In [4]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()


2023-01-14 01:14:08.302673: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-14 01:14:08.349778: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 01:14:08.402619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 01:14:08.404621: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10280748954774505621
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 13953925120
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2039797451821660132
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [5]:
# checking GPU utilization
device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
  raise SystemError('GPU device not found!')

print(f'Found GPU at {device}')


Found GPU at /device:GPU:0


2023-01-14 01:14:42.989119: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 01:14:42.991234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 01:14:42.993019: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 01:14:42.994858: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 01:14:42.996658: I tensorflow/compiler/xla/stream_executo

### Dataset Load
Load the HardHat Dataset zip from drive folder

zip_path : "/content/drive/MyDrive/HardHead Dataset/HardHat_Dataset.zip"

Define unzipping function




In [6]:
def unzipping(src, dst):
  if not os.path.isdir(dst):
    os.mkdir(dst) # unzip path

  # unzipping to dst
  with zipfile.ZipFile(src, 'r') as zip_ref:
    zip_ref.extractall(dst)

In [ ]:

zip_path = './HardHead_Dataset/HardHat_Dataset.zip'
dataset_path = 'Hardhat_dataset'

# unzip
unzipping(zip_path, dataset_path)

## Detect Hard Hat

In [7]:
def detect_img(path):
  num_classes = 2
  classes = class_dict
  weights = '/content/drive/MyDrive/HardHead_Dataset/checkpoints/yolov3_train_6.tf'

  # Lower threshold due to insufficient training
  yolo_iou_threshold = 0.6
  yolo_score_threshold = 0.5

  #if FLAGS.tiny:
  #    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
  #else:
  hardhead_model = YoloV3(classes=num_classes)

  hardhead_model.load_weights(weights).expect_partial()
  logging.info('weights loaded')

  #class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
  #logging.info('classes loaded')

  img_raw = tf.image.decode_image(
      open(image_demo, 'rb').read(), channels=3)

  img = tf.expand_dims(img_raw, 0)
  img = transform_images(img, image_size)

  t1 = time.time()
  boxes, scores, classes, nums = hardhead_model(img)
  t2 = time.time()
  #logging.info('time: {}'.format(t2 - t1))

  #logging.info('detections:')


  for i in range(nums[0]):
      logging.info('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                          np.array(scores[0][i]),
                                          np.array(boxes[0][i])))

  img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
  img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

  from IPython.display import Image, display
  display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))
